In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:

import torch
print(torch.cuda.is_available())

True


In [3]:
from transformers import AutoModelForCausalLM,AutoTokenizer
import torch
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

In [4]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bigscience/bloom-1b7")
model = AutoModelForSequenceClassification.from_pretrained("bigscience/bloom-1b7", num_labels=3)

tokenizer_config.json:   0%|          | 0.00/222 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/715 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

Some weights of BloomForSequenceClassification were not initialized from the model checkpoint at bigscience/bloom-1b7 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
for name, param in model.named_parameters():
        param.requires_grad = False

In [6]:
for name, param in model.named_parameters():
    if "transformer.h.23" in name:
        param.requires_grad = True

In [7]:
for name, param in model.named_parameters():
    print(f"{name}: requires_grad={param.requires_grad}")

transformer.word_embeddings.weight: requires_grad=False
transformer.word_embeddings_layernorm.weight: requires_grad=False
transformer.word_embeddings_layernorm.bias: requires_grad=False
transformer.h.0.input_layernorm.weight: requires_grad=False
transformer.h.0.input_layernorm.bias: requires_grad=False
transformer.h.0.self_attention.query_key_value.weight: requires_grad=False
transformer.h.0.self_attention.query_key_value.bias: requires_grad=False
transformer.h.0.self_attention.dense.weight: requires_grad=False
transformer.h.0.self_attention.dense.bias: requires_grad=False
transformer.h.0.post_attention_layernorm.weight: requires_grad=False
transformer.h.0.post_attention_layernorm.bias: requires_grad=False
transformer.h.0.mlp.dense_h_to_4h.weight: requires_grad=False
transformer.h.0.mlp.dense_h_to_4h.bias: requires_grad=False
transformer.h.0.mlp.dense_4h_to_h.weight: requires_grad=False
transformer.h.0.mlp.dense_4h_to_h.bias: requires_grad=False
transformer.h.1.input_layernorm.weight: 

In [8]:
from datasets import load_dataset

In [9]:
dataset_en = load_dataset("xnli", "en")
dataset_sw = load_dataset("xnli", "sw")

README.md:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/50.2M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/308k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/157k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/392702 [00:00<?, ? examples/s]

...

test-00000-of-00001.parquet:   0%|          | 0.00/312k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/158k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/392702 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5010 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2490 [00:00<?, ? examples/s]

In [10]:
def tokenize_function(example):
    return tokenizer(
        text=example["premise"],
        text_pair=example["hypothesis"],
        truncation=True,
        padding="max_length",
        max_length=128,
    )


tokenized_train_en = dataset_en["train"].map(tokenize_function, batched=True)
tokenized_val_en = dataset_en["validation"].map(tokenize_function, batched=True)
tokenized_test_sw = dataset_sw["test"].map(tokenize_function, batched=True)

Map:   0%|          | 0/392702 [00:00<?, ? examples/s]

Map:   0%|          | 0/2490 [00:00<?, ? examples/s]

Map:   0%|          | 0/5010 [00:00<?, ? examples/s]

In [11]:
tokenized_train_en = tokenized_train_en.remove_columns(["premise", "hypothesis"])
tokenized_val_en = tokenized_val_en.remove_columns(["premise", "hypothesis"])
tokenized_test_sw = tokenized_test_sw.remove_columns(["premise", "hypothesis"])

In [12]:
tokenized_train_en.set_format("torch")
tokenized_val_en.set_format("torch")
tokenized_test_sw.set_format("torch")

In [13]:
def compute_metrics(pred):
    predictions, labels = pred
    preds = np.argmax(predictions, axis=1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average="weighted")
    return {"accuracy": acc, "f1": f1}

In [14]:
tokenized_test_sw[99]

{'label': tensor(2),
 'input_ids': tensor([     3,      3,      3,      3,      3,      3,      3,      3,      3,
              3,      3,      3,      3,      3,      3,      3,      3,      3,
              3,      3,      3,      3,      3,      3,      3,      3,      3,
              3,      3,      3,      3,      3,      3,      3,      3,      3,
              3,      3,      3,      3,      3,      3,      3,      3,      3,
              3,      3,      3,      3,      3,      3,      3,      3,      3,
              3,      3,      3,      3,      3,      3,      3,      3,      3,
              3,      3,      3,      3,      3,      3,      3,      3,      3,
              3,      3,      3,      3,      3,      3,      3,      3,      3,
              3,      3,      3,      3,      3,      3,      3,      3,      3,
              3,      3,      3,      3,      3,      3,      3,      3,      3,
              3,      3,      3,      3,      3,      3,  58668,  36421,   

In [15]:
model

BloomForSequenceClassification(
  (transformer): BloomModel(
    (word_embeddings): Embedding(250880, 2048)
    (word_embeddings_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
    (h): ModuleList(
      (0-23): 24 x BloomBlock(
        (input_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (self_attention): BloomAttention(
          (query_key_value): Linear(in_features=2048, out_features=6144, bias=True)
          (dense): Linear(in_features=2048, out_features=2048, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (post_attention_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (mlp): BloomMLP(
          (dense_h_to_4h): Linear(in_features=2048, out_features=8192, bias=True)
          (gelu_impl): BloomGelu()
          (dense_4h_to_h): Linear(in_features=8192, out_features=2048, bias=True)
        )
      )
    )
    (ln_f): LayerNorm((2048,), eps=1e-05, elementwise_affine

In [16]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score


In [17]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01, 
    save_strategy="steps",
    save_steps=10000,
    save_total_limit=3,
    load_best_model_at_end=True,
metric_for_best_model="accuracy",
 resume_from_checkpoint=True, 

)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [18]:
training_args

TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,
eval_on_start=False,
eval_steps=500,
eval_strategy=steps,
eval_use_gather_object=False,
evaluation_strategy=steps,
fp1

In [19]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_en,
    eval_dataset=tokenized_val_en,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

In [21]:
trainer.evaluate(tokenized_val_en)

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: shashankbabu93 (shashankbabu93-iit-delhi). Use `wandb login --relogin` to force relogin


{'eval_loss': 12.569315910339355,
 'eval_model_preparation_time': 0.005,
 'eval_accuracy': 0.306425702811245,
 'eval_f1': 0.24326843976216314,
 'eval_runtime': 128.3534,
 'eval_samples_per_second': 19.4,
 'eval_steps_per_second': 2.431}

In [22]:
trainer.evaluate(tokenized_test_sw)

{'eval_loss': 6.591902256011963,
 'eval_model_preparation_time': 0.005,
 'eval_accuracy': 0.31057884231536925,
 'eval_f1': 0.2511820328463495,
 'eval_runtime': 255.6849,
 'eval_samples_per_second': 19.594,
 'eval_steps_per_second': 2.452}

In [ ]:
trainer.train()

In [21]:
trainer

In [22]:
trainer.evaluate(tokenized_val_en)

{'eval_loss': 0.6939421892166138,
 'eval_accuracy': 0.706425702811245,
 'eval_f1': 0.7081596250960012}

In [23]:
trainer.evaluate(tokenized_test_sw)

{'eval_loss': 2.646334171295166,
 'eval_accuracy': 0.39201596806387223,
 'eval_f1': 0.3183377692399542}